### Get info box

#### Import the libraries 

In [2]:
#from bs4 import BeautifulSoup as bs
import requests

#### Request the page

In [ ]:
r = requests.get('https://en.wikipedia.org/wiki/The_Beatles:_Get_Back#The_Beatles:_Get_Back_%E2%80%93_The_Rooftop_Concert')
moviePageHTML = bs(r.content)
infoBox = moviePageHTML.select('.infobox')
# print(infoBox[0].prettify())

In [3]:
url = 'http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL'
headers = {'user-agent': 'my-app/0.0.1'}
response = requests.get(url, headers=headers)

In [5]:
print(response.text)

ï»¿<?xml version="1.0" encoding="utf-8"?>
<wb:data page="1" pages="325" per_page="50" total="16226" sourceid="2" sourcename="World Development Indicators" lastupdated="2022-02-15" xmlns:wb="http://www.worldbank.org">
  <wb:data>
    <wb:indicator id="SP.POP.TOTL">Population, total</wb:indicator>
    <wb:country id="ZH">Africa Eastern and Southern</wb:country>
    <wb:countryiso3code>AFE</wb:countryiso3code>
    <wb:date>2020</wb:date>
    <wb:value>677243299</wb:value>
    <wb:unit />
    <wb:obs_status />
    <wb:decimal>0</wb:decimal>
  </wb:data>
  <wb:data>
    <wb:indicator id="SP.POP.TOTL">Population, total</wb:indicator>
    <wb:country id="ZH">Africa Eastern and Southern</wb:country>
    <wb:countryiso3code>AFE</wb:countryiso3code>
    <wb:date>2019</wb:date>
    <wb:value>660046272</wb:value>
    <wb:unit />
    <wb:obs_status />
    <wb:decimal>0</wb:decimal>
  </wb:data>
  <wb:data>
    <wb:indicator id="SP.POP.TOTL">Population, total</wb:indicator>
    <wb:country id="ZH">A

In [ ]:
# grab table rows from the infoBox
trows = infoBox[0].find_all('tr')
for tr in trows:
    print(tr.prettify())

In [ ]:
def getInfoValue(rowData):
    if row.find('li'):
        return [li.get_text(" ", strip = True).replace('\xa0', ' ') for li in rowData.find_all('li')]
    else:
        return rowData.get_text(" ", strip = True).replace('\xa0', ' ')

In [ ]:
movieInfo = {}
for index, row in enumerate(trows):
    if index == 0:
        movieInfo['title'] = row.find('th').get_text(" ", strip = True).replace('\xa0', ' ')
    elif index == 1:
        movieInfo['poster'] = 'https:' + row.find('img')['src'].replace('\xa0', ' ')
    else:
        infoKey = row.find('th').get_text(" ", strip = True).replace('\xa0', ' ')
        infoValue = getInfoValue(row.find('td'))
        movieInfo[infoKey] = infoValue

In [ ]:
movieInfo

### Get info boxes for all movies

soup objects are live connections. Changing at any point will reflect on the variables saved from before

In [2]:
def getInfoValue(rowData):
    if rowData:
        if rowData.find('li'):
            return [li.get_text(" ", strip = True).replace('\xa0', ' ') for li in rowData.find_all('li')]
        elif rowData.find('br'):
            return [text for text in rowData.stripped_strings]
        else:
            return rowData.get_text(" ", strip = True).replace('\xa0', ' ')
    else:
        return ''

def clearSups(page):
    supTags = page.find_all(['sup', 'span'])
    for supTag in supTags:
        supTag.decompose()

def get_info_box(url):
    r = requests.get(url)
    moviePageHTML = bs(r.content)
    clearSups(moviePageHTML)
    infoBox = moviePageHTML.select('.infobox')
    if infoBox:
        trows = infoBox[0].find_all('tr')
    else:
        return ''
    
    movieInfo = {}
    for index, row in enumerate(trows):
        if index == 0:
            movieInfo['title'] = row.find('th').get_text(" ", strip = True).replace('\xa0', ' ')
        elif index == 1:
            movieInfo['poster'] = 'https:' + row.find('img')['src'].replace('\xa0', ' ') if row.find('img') else ''
        else:
            infoKey = row.find('th').get_text(" ", strip = True).replace('\xa0', ' ') if row.find('th') else index
            infoValue = getInfoValue(row.find('td'))
#             print(infoValue)
            movieInfo[infoKey] = infoValue
    return movieInfo

In [3]:
r = get_info_box('https://en.wikipedia.org/wiki/Honey,_I_Shrunk_the_Kids')
r

{'title': 'Honey, I Shrunk the Kids',
 'poster': 'https://upload.wikimedia.org/wikipedia/en/thumb/4/4b/Honey_I_Shrunk_the_kids.jpg/220px-Honey_I_Shrunk_the_kids.jpg',
 'Directed by': 'Joe Johnston',
 'Screenplay by': ['Ed Naha', 'Tom Schulman'],
 'Story by': ['Stuart Gordon', 'Brian Yuzna', 'Ed Naha'],
 'Produced by': 'Penney Finkelman Cox',
 'Starring': ['Rick Moranis',
  'Matt Frewer',
  'Marcia Strassman',
  'Kristine Sutherland'],
 'Cinematography': 'Hiro Narita',
 'Edited by': 'Michael A. Stevenson',
 'Music by': 'James Horner',
 'Production companies': ['Walt Disney Pictures',
  'Silver Screen Partners III'],
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Release date': ['June 23, 1989'],
 'Running time': '93 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$18 million',
 'Box office': '$222.7 million'}

In [14]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
moviePageHTML = bs(r.content)
movies = moviePageHTML.select('.wikitable.sortable i a')
# print(len(movies))

movieInfoList  = []
basePath = 'https://en.wikipedia.org/'
for index, movie in enumerate(movies):
    if index % 10 == 0:
        print(f'{index} movies scraped')
    try:
        link = basePath + movie['href']
        title = movie['title']
        movieDetails = get_info_box(link)
        if movieDetails:
            movieInfoList.append(movieDetails)
    except Exception as e:
        print(movie.get_text())
        print(e)

0 movies scraped
10 movies scraped
20 movies scraped
30 movies scraped
40 movies scraped
50 movies scraped
60 movies scraped
70 movies scraped
80 movies scraped
90 movies scraped
100 movies scraped
110 movies scraped
120 movies scraped
130 movies scraped
140 movies scraped
150 movies scraped
160 movies scraped
170 movies scraped
180 movies scraped
190 movies scraped
200 movies scraped
210 movies scraped
220 movies scraped
230 movies scraped
240 movies scraped
250 movies scraped
260 movies scraped
270 movies scraped
280 movies scraped
290 movies scraped
300 movies scraped
310 movies scraped
320 movies scraped
330 movies scraped
340 movies scraped
350 movies scraped
360 movies scraped
370 movies scraped
380 movies scraped
390 movies scraped
400 movies scraped
410 movies scraped
420 movies scraped
430 movies scraped
440 movies scraped
450 movies scraped
460 movies scraped
470 movies scraped
480 movies scraped
490 movies scraped
500 movies scraped
61
'NoneType' object has no attribute 'get

In [15]:
print(len(movieInfoList))
print(len(movies))

522
526


#### Save movie data to JSON

In [44]:
import json

def saveData(title, data):
#   with is like the open method to open a file. f refers to the file object
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii = False, indent = 2)

In [2]:
import json

def loadData(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

In [18]:
saveData('disneyDataFetchCleaned.json', movieInfoList)

In [38]:
movieInfoList2 = loadData('disneyDataFetchCleaned.json')

In [35]:
print(len(movieInfoList2))

522


### Clean the data

#### some ideas after looking at the json object

~~clean up references~~ - handled using the decompose method <br>
~~split up concatenated names~~ - handled using test cases <br>
~~Convert running time~~ - handled using str.split() method <br>
~~Check whether thead values are properly retrieved and stored~~ - handled using test cases <br>
~~dates need to be standardized (Release Date)~~ - handled using datetime <br>
~~Convert Budget & Box Office values to numbers~~ - handled using regex <br>

#### Convert running time

In [39]:
def minutesToInteger(runningTimeString):
    try:
        if runningTimeString == 'N/A':
            return 0
        elif isinstance(runningTimeString, list):
            return int(runningTimeString[0].split(' ')[0])
        else:
            if '–' in runningTimeString.split(' ')[0]:
                return int(runningTimeString.split(' ')[0].split('–')[0])
            else:
                return int(runningTimeString.split(' ')[0])
    except Exception as e:
        print(e)

In [40]:
# convert running time to integers
# print([movie.get('Running time', 'N/A') for movie in movieInfoList])
for movie in movieInfoList2:
    movie['Running Time (int)'] = minutesToInteger(movie.get('Running time', 'N/A'))

#### Convert budget and box office

In [28]:
for movie in movieInfoList2:
    print(f"Title: {movie.get('title', 'N/A')}; Budget: {movie.get('Budget', 'N/A')}; Box Office: {movie.get('Box office', 'N/A')}")

Title: Academy Award Review of; Budget: N/A; Box Office: $45.472
Title: Snow White and the Seven Dwarfs; Budget: $1.49 million; Box Office: $418 million
Title: Pinocchio; Budget: $2.6 million; Box Office: $164 million
Title: Fantasia; Budget: $2.28 million; Box Office: $76.4–$83.3 million (United States and Canada)
Title: The Reluctant Dragon; Budget: $600,000; Box Office: $960,000 (worldwide rentals)
Title: Dumbo; Budget: $950,000; Box Office: >$1.3 million (est. United States/Canada rentals, 1941)
Title: Bambi; Budget: $858,000; Box Office: $267.4 million
Title: Saludos Amigos; Budget: N/A; Box Office: $1.135 million (worldwide rentals)
Title: Victory Through Air Power; Budget: $788,000; Box Office: $799,000
Title: The Three Caballeros; Budget: N/A; Box Office: $3.355 million (worldwide rentals)
Title: Make Mine Music; Budget: $1.35 million; Box Office: $3.275 million (worldwide rentals)
Title: Song of the South; Budget: $2.125 million; Box Office: $65 million
Title: Fun and Fancy Fr

In [41]:
import re
number = r'\d+(,\d{3})*\.*\d*'

# $12.2 million
units = r'thousand|million|billion'
wordAmount = rf'\${number}(–|-|\sto\s)?({number})?\s({units})'

# $790,000
numberAmount = rf'\${number}' 

def getZeros(unit):
    unitDict = {'thousand': 1000, 'million': 1000000, 'billion': 1000000000}
    return unitDict[unit]

def convertValueSyntaxToNumber(syntax):
    valueStringNumber = re.search(number, syntax).group().replace(',', '')
    value = float(valueStringNumber)
    return value

def convertWordSyntaxToNumber(syntax):
    valueStringNumber = re.search(number, syntax).group().replace(',', '')
    valueStringText = re.search(units, syntax, flags=re.I).group().lower()
    value = float(valueStringNumber) * getZeros(valueStringText)
    return value

def money_conversion(money):
    if money == 'N/A':
        return None
    
    # if money passed is a list
    if isinstance(money, list):
        money = money[0]

    # matching the whole expense '$790,000', '$30 million' including the $ symbol
    wordSyntax = re.search(wordAmount, money, flags=re.I)
    valueSyntax = re.search(numberAmount, money)
    # extracting the number out of them
    if wordSyntax:
        print(f'Word matched: {wordSyntax.group()}')
        return convertWordSyntaxToNumber(wordSyntax.group())
    elif valueSyntax:
        print(f'value matched: {valueSyntax.group()}')
        return convertValueSyntaxToNumber(valueSyntax.group())
    else:
        return None

In [43]:
# convert expenses to integers
for movie in movieInfoList2:
    movie['Budget (float)'] = money_conversion(movie.get('Budget', 'N/A'))
    movie['Box office (float)'] = money_conversion(movie.get('Box office', 'N/A'))

value matched: $45.472
Word matched: $1.49 million
Word matched: $418 million
Word matched: $2.6 million
Word matched: $164 million
Word matched: $2.28 million
Word matched: $83.3 million
value matched: $600,000
value matched: $960,000
value matched: $950,000
Word matched: $1.3 million
value matched: $858,000
Word matched: $267.4 million
Word matched: $1.135 million
value matched: $788,000
value matched: $799,000
Word matched: $3.355 million
Word matched: $1.35 million
Word matched: $3.275 million
Word matched: $2.125 million
Word matched: $65 million
Word matched: $3.165 million
Word matched: $1.5 million
Word matched: $2.56 million
Word matched: $1.5 million
Word matched: $3.7 million
Word matched: $1.625 million
Word matched: $2.2 million
Word matched: $182 million
value matched: $1,800,000
value matched: $4,100,000
Word matched: $3 million
Word matched: $2.4 million
Word matched: $2.1 million
Word matched: $4 million
Word matched: $87.4 million
Word matched: $2 million
Word matched

#### Convert release date

In [53]:
saveData('dateConversion.json', movieInfoList2)

In [3]:
movieInfoList3 = loadData('dateConversion.json')

In [74]:
'Release date' == 'Release date'

True

In [7]:
for movie in movieInfoList3:
    print(movie.get('title', 'N/A'))
    print(movie.get('Release date', 'N/A'))
    print(type(movie.get('Release date', 'N/A')))
    print()
#     print(f"Title: {movie.get('title', 'N/A')}; Release Date: {movie.get('Release date', 'N/A')}; Release Dates: {movie.get('Release dates', 'N/A')}")

Academy Award Review of
['May 19, 1937']
<class 'list'>

Snow White and the Seven Dwarfs
['December 21, 1937 ( Carthay Circle Theatre )']
<class 'list'>

Pinocchio
['February 7, 1940 ( Center Theatre )', 'February 23, 1940 (United States)']
<class 'list'>

Fantasia
['November 13, 1940']
<class 'list'>

The Reluctant Dragon
['June 27, 1941']
<class 'list'>

Dumbo
['October 23, 1941 (New York City)', 'October 31, 1941 (U.S.)']
<class 'list'>

Bambi
['August 9, 1942 (World Premiere – London)', 'August 13, 1942 (Premiere – New York City)', 'August 21, 1942 (U.S.)']
<class 'list'>

Saludos Amigos
['August 24, 1942 (World Premiere – Rio de Janeiro)', 'February 6, 1943 (U.S. Premiere – Boston)', 'February 19, 1943 (U.S.)']
<class 'list'>

Victory Through Air Power
['July 17, 1943']
<class 'list'>

The Three Caballeros
['December 21, 1944 (Mexico City)', 'February 3, 1945 (US)']
<class 'list'>

Make Mine Music
['April 20, 1946 (New York City premiere)', 'August 15, 1946 (U.S.)']
<class 'list'>

In [5]:
# we have two key names: Release Date; Release Dates. we have to merge them into one
for movie in movieInfoList3:
    print(movie)
    print(f"Release Date: {movie.get('Release date', 'N/A')}; Release Dates: {movie.get('Release dates', 'N/A')}")
    if 'Release dates' in movie.keys():
        movie['Release date'] = movie['Release dates']
        del movie['Release dates']

{'title': 'Academy Award Review of', 'poster': 'https://upload.wikimedia.org/wikipedia/en/thumb/7/7a/Aarfvdsf.jpg/220px-Aarfvdsf.jpg', 'Production company': 'Walt Disney Productions', 'Distributed by': 'RKO Radio Pictures', 'Release date': ['May 19, 1937'], 'Running time': '41 minutes (74 minutes 1966 release)', 'Country': 'United States', 'Language': 'English', 'Box office': '$45.472', 'Running Time (int)': 41, 'Budget (float)': None, 'Box office (float)': 45.472}
Release Date: ['May 19, 1937']; Release Dates: N/A
{'title': 'Snow White and the Seven Dwarfs', 'poster': 'https://upload.wikimedia.org/wikipedia/en/thumb/4/49/Snow_White_1937_poster.png/220px-Snow_White_1937_poster.png', 'Directed by': ['David Hand', 'William Cottrell', 'Wilfred Jackson', 'Larry Morey', 'Perce Pearce', 'Ben Sharpsteen'], 'Written by': ['Ted Sears', 'Richard Creedon', 'Otto Englander', 'Dick Rickard', 'Earl Hurd', 'Merrill De Maris', 'Dorothy Ann Blank', 'Webb Smith'], 'Based on': ['Snow White', 'by The', 'B

In [17]:
# we have 4 formats: '%B %d, %Y', '%d %B %Y', '%Y', '%B %Y'
from datetime import datetime
# releaseDates = [movie.get('Release date', 'N/A') for movie in movieInfoList3]

def cleanDate(date):
    return date.split('(')[0].strip()

def convertDate(date):
    if isinstance(date, list):
        date = date[0]
    elif date == 'N/A':
        return None
    date = cleanDate(date)
    frmts = ['%B %d, %Y', '%d %B %Y', '%Y', '%B %Y']
    for frmt in frmts:
        try:
            return datetime.strptime(date, frmt)
        except:
            pass
    return None

In [18]:
for movie in movieInfoList3:
    movie['Release date (datetime)'] = convertDate(movie.get('Release date', 'N/A'))

In [19]:
movieInfoList3[157]

{'title': 'Tex',
 'poster': 'https://upload.wikimedia.org/wikipedia/en/thumb/c/cb/Poster_of_Tex_%28film%29.jpg/220px-Poster_of_Tex_%28film%29.jpg',
 'Directed by': 'Tim Hunter',
 'Screenplay by': ['Charles S. Haas', 'Tim Hunter'],
 'Based on': ['Tex', 'by', 'S. E. Hinton'],
 'Produced by': ['Tim Zinnemann', 'Ron Miller'],
 'Starring': ['Matt Dillon',
  'Jim Metzler',
  'Meg Tilly',
  'Bill McKinney',
  'Ben Johnson'],
 'Cinematography': 'Ric Waite',
 'Edited by': 'Howard E. Smith',
 'Music by': 'Pino Donaggio',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': 'July 30, 1982',
 'Running time': '103 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$5 million',
 'Box office': '$7.4 million',
 'Running Time (int)': 103,
 'Budget (float)': 5000000.0,
 'Box office (float)': 7400000.0,
 'Release date (datetime)': datetime.datetime(1982, 7, 30, 0, 0)}

In [21]:
# cannot save as a JSON file as it doesnt support datetime objects
# saveData('afterDatetime.json', movieInfoList3)

TypeError: Object of type datetime is not JSON serializable

In [37]:
import pickle

def saveDataWithPickle(data, fileName):
    with open(fileName, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
saveDataWithPickle(movieInfoList3, 'afterDatetime.pickle')

In [6]:
import pickle
def loadDataWithPickle(fileName):
    with open(fileName, 'rb') as handle:
        return pickle.load(handle)

In [7]:
movieInfoListAfterDatetime = loadDataWithPickle('afterDatetime.pickle')

In [31]:
len(movieInfoListAfterDatetime)

522

### Fetch IMDB, Rotten Tomatoes, Metascore scores (using OMDB API)

In [29]:
import requests
# import os # use windows environment variables to store sensitive information and then use in python
import urllib # use urllib to encode url for api fetches


def fetchOMDBInfo(title):
    baseURL = 'http://www.omdbapi.com/?'
    parameters = {
        "apikey": "fbf4ea75", 
        't': title
    }
    encodedParameters = urllib.parse.urlencode(parameters)
    link = baseURL + encodedParameters
    print(link)
    return requests.get(link).json()

def getRottenTomatoScore(omdbInfo):
    ratings = omdbInfo.get('Ratings', [])
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
    return None

In [25]:
pinocchio = fetchOMDBInfo('Pinocchio')

http://www.omdbapi.com/?apikey=fbf4ea75&t=Pinocchio


In [28]:
pinocchio

{'Title': 'Pinocchio',
 'Year': '1940',
 'Rated': 'G',
 'Released': '23 Feb 1940',
 'Runtime': '88 min',
 'Genre': 'Animation, Adventure, Comedy',
 'Director': 'Norman Ferguson, T. Hee, Wilfred Jackson',
 'Writer': 'Carlo Collodi, Ted Sears, Otto Englander',
 'Actors': 'Dickie Jones, Christian Rub, Mel Blanc',
 'Plot': 'A living puppet, with the help of a cricket as his conscience, must prove himself worthy to become a real boy.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': 'Won 2 Oscars. 7 wins total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTU4Mzk3ODIyOF5BMl5BanBnXkFtZTgwODgyNzk2NjE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.4/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '100%'},
  {'Source': 'Metacritic', 'Value': '99/100'}],
 'Metascore': '99',
 'imdbRating': '7.4',
 'imdbVotes': '139,393',
 'imdbID': 'tt0032910',
 'Type': 'movie',
 'DVD': '03 Dec 2002',
 'BoxOffice': '$84,254,167',
 'Production': 'N/A',
 'Webs

In [30]:
getRottenTomatoScore(pinocchio)

'100%'

In [31]:
for movie in movieInfoListAfterDatetime:
    title = movie['title']
    omdbInfo = fetchOMDBInfo(title)
    movie['imdb'] = omdbInfo.get('imdbRating', 'N/A')
    movie['imdbVotes'] = omdbInfo.get('imdbVotes', 'N/A')
    movie['metascore'] = omdbInfo.get('Metascore', 'N/A')
    movie['rottenTomatoes'] = getRottenTomatoScore(omdbInfo)

http://www.omdbapi.com/?apikey=fbf4ea75&t=Academy+Award+Review+of
http://www.omdbapi.com/?apikey=fbf4ea75&t=Snow+White+and+the+Seven+Dwarfs
http://www.omdbapi.com/?apikey=fbf4ea75&t=Pinocchio
http://www.omdbapi.com/?apikey=fbf4ea75&t=Fantasia
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Reluctant+Dragon
http://www.omdbapi.com/?apikey=fbf4ea75&t=Dumbo
http://www.omdbapi.com/?apikey=fbf4ea75&t=Bambi
http://www.omdbapi.com/?apikey=fbf4ea75&t=Saludos+Amigos
http://www.omdbapi.com/?apikey=fbf4ea75&t=Victory+Through+Air+Power
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Three+Caballeros
http://www.omdbapi.com/?apikey=fbf4ea75&t=Make+Mine+Music
http://www.omdbapi.com/?apikey=fbf4ea75&t=Song+of+the+South
http://www.omdbapi.com/?apikey=fbf4ea75&t=Fun+and+Fancy+Free
http://www.omdbapi.com/?apikey=fbf4ea75&t=Melody+Time
http://www.omdbapi.com/?apikey=fbf4ea75&t=So+Dear+to+My+Heart
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Adventures+of+Ichabod+and+Mr.+Toad
http://www.omdbapi.com/?apikey=fbf

http://www.omdbapi.com/?apikey=fbf4ea75&t=A+Tale+of+Two+Critters
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Rescuers
http://www.omdbapi.com/?apikey=fbf4ea75&t=Herbie+Goes+to+Monte+Carlo
http://www.omdbapi.com/?apikey=fbf4ea75&t=Pete%27s+Dragon
http://www.omdbapi.com/?apikey=fbf4ea75&t=Candleshoe
http://www.omdbapi.com/?apikey=fbf4ea75&t=Return+from+Witch+Mountain
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Cat+from+Outer+Space
http://www.omdbapi.com/?apikey=fbf4ea75&t=Hot+Lead+and+Cold+Feet
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+North+Avenue+Irregulars
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Apple+Dumpling+Gang+Rides+Again
http://www.omdbapi.com/?apikey=fbf4ea75&t=Unidentified+Flying+Oddball
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Black+Hole
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Omega+Connection
http://www.omdbapi.com/?apikey=fbf4ea75&t=Midnight+Madness
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Watcher+in+the+Woods
http://www.omdbapi.com/?apikey=fbf4

http://www.omdbapi.com/?apikey=fbf4ea75&t=Treasure+Planet
http://www.omdbapi.com/?apikey=fbf4ea75&t=101+Dalmatians+II%3A
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Jungle+Book+2
http://www.omdbapi.com/?apikey=fbf4ea75&t=Inspector+Gadget+2
http://www.omdbapi.com/?apikey=fbf4ea75&t=Piglet%27s+Big+Movie
http://www.omdbapi.com/?apikey=fbf4ea75&t=Ghosts+of+the+Abyss
http://www.omdbapi.com/?apikey=fbf4ea75&t=Holes
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Lizzie+McGuire+Movie
http://www.omdbapi.com/?apikey=fbf4ea75&t=Finding+Nemo
http://www.omdbapi.com/?apikey=fbf4ea75&t=Pirates+of+the+Caribbean%3A+The+Curse+of+the+Black+Pearl
http://www.omdbapi.com/?apikey=fbf4ea75&t=Freaky+Friday
http://www.omdbapi.com/?apikey=fbf4ea75&t=Stitch%21+The+Movie
http://www.omdbapi.com/?apikey=fbf4ea75&t=George+of+the+Jungle+2
http://www.omdbapi.com/?apikey=fbf4ea75&t=Brother+Bear
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Haunted+Mansion
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Young+Black+Stalli

http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Odd+Life+of+Timothy+Green
http://www.omdbapi.com/?apikey=fbf4ea75&t=Secret+of+the+Wings
http://www.omdbapi.com/?apikey=fbf4ea75&t=Beverly+Hills+Chihuahua+3%3A+Viva+la+Fiesta%21
http://www.omdbapi.com/?apikey=fbf4ea75&t=Frankenweenie
http://www.omdbapi.com/?apikey=fbf4ea75&t=Wreck-It+Ralph
http://www.omdbapi.com/?apikey=fbf4ea75&t=Oz+the+Great+and+Powerful
http://www.omdbapi.com/?apikey=fbf4ea75&t=Wings+of+Life
http://www.omdbapi.com/?apikey=fbf4ea75&t=Monsters+University
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Lone+Ranger
http://www.omdbapi.com/?apikey=fbf4ea75&t=Planes
http://www.omdbapi.com/?apikey=fbf4ea75&t=Super+Buddies
http://www.omdbapi.com/?apikey=fbf4ea75&t=Frozen
http://www.omdbapi.com/?apikey=fbf4ea75&t=Saving+Mr.+Banks
http://www.omdbapi.com/?apikey=fbf4ea75&t=The+Pirate+Fairy
http://www.omdbapi.com/?apikey=fbf4ea75&t=Muppets+Most+Wanted
http://www.omdbapi.com/?apikey=fbf4ea75&t=Bears
http://www.omdbapi.com/?apikey=fbf4ea7

In [43]:
movieInfoListAfterDatetime[87]

{'title': 'Charlie, the Lonesome Cougar',
 'poster': 'https://upload.wikimedia.org/wikipedia/en/thumb/c/ca/Charlie%2C_the_Lonesome_Cougar_poster.jpg/220px-Charlie%2C_the_Lonesome_Cougar_poster.jpg',
 'Directed by': 'Winston Hibler',
 'Written by': 'Jack Speirs',
 'Produced by': ['Walt Disney', 'Winston Hibler', 'Charles L. Draper'],
 'Starring': ['Ron Brown',
  'Brian Russell',
  'Linda Wallace',
  'Jim Wilson',
  'Clifford Peterson'],
 'Narrated by': 'Rex Allen',
 'Cinematography': ['William Bacon III', 'Lloyd Beebe'],
 'Edited by': 'Gregg McLaughlin',
 'Music by': ['Franklyn Marks', 'Speirs Marks'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['October 19, 1967'],
 'Running time': '75 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Running Time (int)': 75,
 'Budget (float)': None,
 'Box office (float)': None,
 'Release date (datetime)': datetime.datetime(1967, 10, 19, 0, 0),
 'imdb': '6.8',
 'imdbV

### Save the datasets

#### Pickling 

In [38]:
saveDataWithPickle(movieInfoListAfterDatetime, 'finalWaltDisneyFilms.pickle')

#### To JSON

In [39]:
moviesWDJSONVer = [movie.copy() for movie in movieInfoListAfterDatetime]

In [41]:
for movie in moviesWDJSONVer:
    currMovieDate = movie['Release date (datetime)']
    if currMovieDate:
        movie['Release date (datetime)'] = currMovieDate.strftime('%B %d, %Y')
    else:
        movie['Release date (datetime)'] = None

In [42]:
moviesWDJSONVer[87]

{'title': 'Charlie, the Lonesome Cougar',
 'poster': 'https://upload.wikimedia.org/wikipedia/en/thumb/c/ca/Charlie%2C_the_Lonesome_Cougar_poster.jpg/220px-Charlie%2C_the_Lonesome_Cougar_poster.jpg',
 'Directed by': 'Winston Hibler',
 'Written by': 'Jack Speirs',
 'Produced by': ['Walt Disney', 'Winston Hibler', 'Charles L. Draper'],
 'Starring': ['Ron Brown',
  'Brian Russell',
  'Linda Wallace',
  'Jim Wilson',
  'Clifford Peterson'],
 'Narrated by': 'Rex Allen',
 'Cinematography': ['William Bacon III', 'Lloyd Beebe'],
 'Edited by': 'Gregg McLaughlin',
 'Music by': ['Franklyn Marks', 'Speirs Marks'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['October 19, 1967'],
 'Running time': '75 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Running Time (int)': 75,
 'Budget (float)': None,
 'Box office (float)': None,
 'Release date (datetime)': 'October 19, 1967',
 'imdb': '6.8',
 'imdbVotes': '855',
 'met

In [45]:
saveData('moviesWDJSONVer.json', moviesWDJSONVer)

#### To CSV

In [47]:
import pandas as pd
df = pd.DataFrame(movieInfoListAfterDatetime)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Columns: 141 entries, title to Theme park attraction(s)
dtypes: datetime64[ns](1), float64(2), int64(1), object(137)
memory usage: 575.1+ KB
